In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import keras
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as k
from keras.applications.vgg16 import decode_predictions
from keras.utils.np_utils import to_categorical
import h5py

Using TensorFlow backend.


In [2]:
os.chdir('/home/ankit/Downloads/train-scene classification')
os.getcwd()

labels = pd.read_csv('train.csv')
images =list(labels['image_name'])
Y_train = np.array(list(labels['label'])).reshape(-1, 1)
print(Y_train.shape)

labels_test = pd.read_csv('test.csv')
images_test =list(labels_test['image_name'])

os.chdir('/home/ankit/Downloads/train-scene classification/train')
train_img=[]
for img in images:

    temp_img=image.load_img(img,target_size=(128,128))

    temp_img=image.img_to_array(temp_img)

    train_img.append(temp_img)

train_img=np.array(train_img) 
train_img=preprocess_input(train_img)
print(train_img.shape)



(17034, 1)
(17034, 128, 128, 3)


In [3]:
test_img=[]
for img in images_test:

    temp_img=image.load_img(img,target_size=(128,128))

    temp_img=image.img_to_array(temp_img)

    test_img.append(temp_img)

test_img=np.array(test_img) 
test_img=preprocess_input(test_img)
print(test_img.shape)

(7301, 64, 64, 3)


In [4]:
input_tensor = Input(shape=(128, 128, 3))
vgg_model = VGG16(weights='imagenet', include_top = False, input_tensor =input_tensor)
vgg_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0         
__________

In [5]:
num_classes = 6

layer_dict = dict([(layer.name, layer) for layer in vgg_model.layers])
x = layer_dict['block2_pool'].output

x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D(pool_size=(2, 2))(x)
x = Dropout(0.25)(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

image_model = Model(input = vgg_model.input, output = x)

for layer in image_model.layers[:7]:
    layer.trainable = False
    
image_model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Y = keras.utils.to_categorical(Y_train, num_classes)

image_model.fit(train_img, Y,
          batch_size=16,
          epochs=10)

score = model.evaluate(train_img, Y)
print('Train loss:', score[0])
print('Train accuracy:', score[1])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  


NameError: name 'Y_train' is not defined